In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.ndimage import gaussian_filter1d as gf


In [2]:
file     = uproot.open("/Users/ishiwata/grams/data/20230620/liquidN_HV174_tr56_bl129000087.bin.root")

header   = file['header']
tree     = file['tree']
QDC      = tree.arrays(['qdc'], library='numpy')['qdc']
waveform = tree.arrays(['waveform'], library='numpy')['waveform']
timebin_width = header.arrays(['timebin_width'], library='numpy')['timebin_width'] #ns
wave_num = tree.arrays(['wave_num'], library='numpy')['wave_num'][0]
time_array    = np.arange(wave_num) * timebin_width #* ns2us

In [3]:
#FFTで周期的な成分を除去
num=len(waveform)
fft_wave=np.zeros((len(waveform),1000)) #fftでpeakを除去した波形
for i in range(len(waveform)):
    offset=abs(np.min(waveform[i]))
    X=waveform[i]
    N =len(X)      #データ長
    fs=1e+9          #サンプリング周波数
    dt =1/fs       # サンプリング間隔
    t = np.arange(0.0, N*dt, dt) #時間軸
    freq = np.linspace(0, fs,N) #周波数軸
    fn=1/dt/2     #ナイキスト周波数
    F=np.fft.fft(X)/(N/2)
    F[(freq>fn)]=0 #ナイキスト周波数以降をカット
    X_0=np.abs(F)
    threshold=np.amax(X_0)*0.03
    peak_index=np.where(X_0 > threshold) #threshold以上のpeak_indexを抽出
    peak_freq=[]
    for n in peak_index[0]:
        if freq[n] > 1e7:
            peak_freq.append(freq[n])

    for hz in peak_freq:
        F[(freq==hz)]=0
    X_1=np.abs(F) #trendを除去した波形(横軸周波数)
    X_2=np.real(np.fft.ifft(F))*N #trendを除去した波形(横軸時間[ns]) waveform

    fft_wave[i]=X_2

In [4]:
##########################################################
#ここにAD converter補正が入る
##########################################################

#baseline補正
blcorr_wave=np.zeros((num,1000))
for i in range(num):
    baseline=np.average(fft_wave[i][0:150])
    blcorr_wave[i]=fft_wave[i]-baseline

In [ ]:
#peak位置補正

# fil1というフラグを作成 
# 0:peak indexが200-400nsの中
# 1:peak indexが200-400nsの外
num=20
fil1=np.zeros(num)
peak_index=np.zeros(num)
gfil_wave=np.zeros((num,1000))
for i in range(num):
    gfil_wave[i]=gf(fft_wave[i],sigma=3,mode="wrap",)
    # peak_indexfft.append(np.argmax(fft_wave[i]))
    peak_index[i]=np.argmax(gfil_wave[i])
    if (peak_index[i] > 200) and (peak_index[i] < 400):
        fil1[i]=0
    else :
        fil1[i]=1
# peak位置をずらして0埋めする
# その後[0:1000]で抽出する
shift_wave=np.zeros((num,1000))
for i in range(num):
    if fil1[i] == 0:
        dif = np.max(peak_index)-peak_index[i] #一番右側にあるpeak_indexとのずれ
        shift=np.zeros(dif.astype("int"))
        shifted_wave=np.hstack((shift,fft_wave[i]))
        shift_wave[i]=shifted_wave[0:1000]